In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
from shapely import geos
from shapely.geometry import Point
import fiona
import matplotlib.pyplot as plt
from fiona.crs import from_epsg,from_string

In [232]:
data = pd.read_csv('../data/' + 'MissingMigrants-Global-2019-12-06T12-34-22.csv')

In [233]:
# extract latitude and longtitude and transform to type float64 
data['lat']=data['Location Coordinates'].astype('str').apply(lambda x: x.split(',')).map(lambda x: x[0]).astype('float64') 
data['long']=data['Location Coordinates'].astype('str').apply(lambda x: x.split(',')).map(lambda x: x[-1]).astype('float64')

In [234]:
data2 = data[['Location Coordinates','Region of Incident','Reported Year','Total Dead and Missing','lat','long','Cause of Death']]

In [235]:
data2geometry = [Point(xy) for xy in zip(data2['long'], data2['lat'])]

In [236]:
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(data2, crs=crs, geometry=data2geometry)

In [237]:
bol_list = (gdf['Reported Year']>=2014)  # To extract all the data after 2017
gdf2014_2019= gdf[bol_list]

In [238]:
import folium
import pandas as pd

# define the world map
world_map = folium.Map()

In [241]:
len(gdf2014_2019)

6201

In [240]:
gdf2014_2019 = gdf2014_2019.dropna(how = 'any')

In [242]:
labels = gdf2014_2019['Region of Incident'].str.cat(gdf2014_2019['Cause of Death'], sep = '<br></br> Cause of death : ')

In [243]:
labels = labels.str.cat(gdf2014_2019['Total Dead and Missing'],sep = '<br></br>Total Dead and Missing:')

In [245]:
from folium import plugins
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(world_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, long,label in zip(gdf2014_2019.lat, gdf2014_2019.long,labels):
    folium.Marker([lat, long],popup=label).add_to(incidents)

# add incidents to map
world_map_new = world_map.add_child(incidents)

# As the data is too big to show directly on notebook, save it as html file to check it through browser
world_map_new.save('world_map_2014_2019.html') 

In [ ]:
Interactive Map, concluding 2014-2019
